In [21]:
from lxml import etree
import abc
import re
import requests

import gensim.models as models
import gensim.models.word2vec as word2vec
import secure

import search
reload(search)
import rewriter
reload(rewriter)


In [1]:
corpus = word2vec.Text8Corpus(secure.DATASET_PATH_BASE + 'enwik8')
# self.model = word2vec.Word2Vec(corpus, workers=8)
# self.model.save(self.MODEL_PATH)

In [9]:
from gensim.models.phrases import *

ph = Phrases(corpus)

In [12]:
phr = Phraser(ph)

In [13]:
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']

In [14]:
print phr[sent]

[u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']


In [17]:
phr[[u'chemical',u'reactions',u'are',u'cool']]

[u'chemical_reactions', u'are', u'cool']

In [21]:
import warnings

In [22]:
import re
warnings.filterwarnings("ignore", message=re.escape("For a faster implementation, use the gensim.models.phrases.Phraser class"))

In [23]:
mod = word2vec.Word2Vec(ph[corpus],size=10)

In [26]:
mod.most_similar(positive=['person'], topn=10)

[(u'looking_forward', 0.9501240849494934),
 (u'compounds]', 0.9304075241088867),
 (u"'''[[Internet", 0.9185891151428223),
 (u'Marks]]', 0.9161316156387329),
 (u'(1995).', 0.9128404259681702),
 (u'anti-Communist', 0.9109591245651245),
 (u'Font', 0.9097691774368286),
 (u'[[National_Library', 0.9085487723350525),
 (u"'''F'''", 0.9075831174850464),
 (u'(Istat_2005)', 0.9073328971862793)]

In [27]:
from gensim.models.wrappers import FastText

In [33]:
# WARNING: takes like 10 min to train
# made from https://github.com/facebookresearch/fastText
path = '/Users/neel/Dropbox/Harvard/Classes/research/query-rewriter/fastText/fasttext'
fm = FastText.train(path, corpus_file=secure.DATASET_PATH_BASE + 'enwik8')

In [61]:
fm.similar_by_word('new_york', topn=20)

[(u'Zork', 0.816493034362793),
 (u'Ankh-Morpork', 0.7960439920425415),
 (u'Artwork', 0.7774671316146851),
 (u'Network', 0.7542241811752319),
 (u'pork', 0.7466747760772705),
 (u'Clockwork', 0.7459302544593811),
 (u'homework', 0.7433512806892395),
 (u'Cork', 0.7420310974121094),
 (u'newsreel', 0.7417821884155273),
 (u'newline', 0.7417464256286621),
 (u'Pork', 0.7369546294212341),
 (u'(Network', 0.729056715965271),
 (u'company_slogan', 0.7259443998336792),
 (u'news]', 0.7191295623779297),
 (u'network', 0.7173770666122437),
 (u'new', 0.7120053172111511),
 (u'Yorkshire', 0.7098603248596191),
 (u'stonework', 0.7096399068832397),
 (u'new,', 0.7078135013580322),
 (u'artwork', 0.7072222232818604)]

In [4]:
from gensim.models.wrappers import FastText
import secure

In [64]:
fm.most_similar(positive=['new_york'], topn=20)

[(u'Zork', 0.816493034362793),
 (u'Ankh-Morpork', 0.7960439920425415),
 (u'Artwork', 0.7774671316146851),
 (u'Network', 0.7542241811752319),
 (u'pork', 0.7466747760772705),
 (u'Clockwork', 0.7459302544593811),
 (u'homework', 0.7433512806892395),
 (u'Cork', 0.7420310974121094),
 (u'newsreel', 0.7417821884155273),
 (u'newline', 0.7417464256286621),
 (u'Pork', 0.7369546294212341),
 (u'(Network', 0.729056715965271),
 (u'company_slogan', 0.7259443998336792),
 (u'news]', 0.7191295623779297),
 (u'network', 0.7173770666122437),
 (u'new', 0.7120053172111511),
 (u'Yorkshire', 0.7098603248596191),
 (u'stonework', 0.7096399068832397),
 (u'new,', 0.7078135013580322),
 (u'artwork', 0.7072222232818604)]

In [ ]:
[]

In [68]:
fm.save(secure.MODEL_PATH_BASE+'fasttext/fasttext')

In [5]:
fm2 = FastText.load(secure.MODEL_PATH_BASE+'fasttext/fasttext')

In [6]:
fm2.similar_by_word('chemistry', topn=25)

[(u'biochemistry', 0.9830586910247803),
 (u'chemistry|', 0.9796395897865295),
 (u'(chemistry)', 0.9524341821670532),
 (u'chemistry,', 0.9513406753540039),
 (u'chemistry==', 0.9481360912322998),
 (u'biochemistry,', 0.9481123685836792),
 (u"'Radiochemistry", 0.9445796608924866),
 (u'Biochemistry', 0.9373379945755005),
 (u'Electrochemistry', 0.9102224111557007),
 (u"chemistry'''", 0.9014816284179688),
 (u'chemist', 0.8953343629837036),
 (u'chemistry.', 0.8936431407928467),
 (u'[[biochemistry|biochemical]]', 0.8890479803085327),
 (u'electrochemistry', 0.8872667551040649),
 (u'biochemistry.', 0.8862274289131165),
 (u'chemist,', 0.8851226568222046),
 (u'biochemist', 0.8840389251708984),
 (u'biochemist,', 0.8771650195121765),
 (u'[[chemistry|chemical]]', 0.8727388381958008),
 (u'Chemistry', 0.8726977109909058),
 (u'ischemic', 0.8639161586761475),
 (u'alchemist', 0.8624183535575867),
 (u'physiology', 0.8568892478942871),
 (u'Chemistry,', 0.8566870093345642),
 (u'[[electrochemistry|electrochemi

In [77]:
fm2.init_ngrams()

IndexError: index 1962073 is out of bounds for size 1474803

In [7]:
from lxml import etree
import abc
import re
import requests

import gensim.models as models
import gensim.models.word2vec as word2vec
import secure

In [16]:
corpus = word2vec.LineSentence(secure.DATASET_PATH_BASE + 'wikiclean8')

In [17]:
model = word2vec.Word2Vec(corpus, workers=8)

In [18]:
MODEL_PATH = secure.MODEL_PATH_BASE+'word2vec/word2vec'

In [19]:
model.save(MODEL_PATH)

In [25]:
import rewriter
reload(rewriter)

w2v = rewriter.Word2VecRewriter(corpus=corpus, create=False)

In [37]:
w2v.rewrite("ohio")

[u'michigan',
 u'iowa',
 u'wisconsin',
 u'illinois',
 u'pennsylvania',
 u'birmingham',
 u'oregon',
 u'arizona',
 u'nebraska',
 u'kentucky',
 u'ohio']

In [46]:
from gensim.models.phrases import *

phrases = Phrases(corpus)

In [ ]:
import re
warnings.filterwarnings("ignore", message=re.escape("For a faster implementation, use the gensim.models.phrases.Phraser class"))

In [ ]:
model = word2vec.Word2Vec(phrases[corpus])

In [ ]:
# warning: you get a lot of gibberish! :(
model.similar_by_word("new_york")

In [ ]:
phr = Phraser(phrases)